In [1]:
import numpy as np
import cv2

K = np.array([[1.06314155e+03, 0.00000000e+00, 9.62163918e+02],
              [0.00000000e+00, 1.06450171e+03, 5.40462082e+02],
              [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
D = np.array([[0.04147462, -0.03934306, -0.00013871, -0.00071449, -0.00889901]])


# trovate usando in calibrazione l'immagine non antidistorta
R = np.array([[ 0.99981412, -0.01682312, -0.00941902],
              [ 0.01644207,  0.99909738, -0.03916732],
              [ 0.01006943,  0.03900517,  0.99918827]])
t = np.array([[-33.05102197], [-15.29010071], [41.22994723]])

# trovate usando in calibrazione l'immagine antidistorta
Rd = np.array([[ 0.99983156, -0.01274002, -0.01321169],
               [ 0.01235797,  0.99951436, -0.02860628],
               [ 0.01356972,  0.02843819,  0.99950344]])
td = np.array([[-34.2646834], [-15.84329211], [42.18894322]])


def px2meters(pt, K, R, t):
    ''' Funzione di conversione tra pixel e metri per ottenere la posizione
    corretta in metri della posizione rilevata nell'immagine RGB-D. La Kinect
    prende (x,y) dall'RGB e (z) dal sensore di profondita', quindi deve essere
    calibrato per forza. Calibrazione fatta in MATLAB! Riporto qui le matrici '''

    # pt e' px
    
    K2 = np.linalg.inv(K)
    R2 = np.linalg.inv(R)
    pt = pt.T

    S = K2.dot(pt)
    N = S - t
    PT = R2.dot(N)
    XYZ = PT # in cm
    print('Point is: ' + str(XYZ))

    return XYZ

In [2]:
reference, _ = cv2.projectPoints(np.array([[0.0, 0.0, 0.0]]), R, t, K, D)
reference = reference.flatten()
print('Reference is: ' + str(reference))
reference = np.array([[int(round(reference[0],0)), int(round(reference[1],0)), 1.0]]) #px coordinates

Reference is: [104.69738764 143.43175586]


In [50]:
img = cv2.imread('/home/optolab/ur2020_workspace/src/telemove/src/example.png') 
# immagine di esempio non antidistorta
# flip e cut per il mio setup
img = cv2.flip(img, 0)
img = img[0:900, 520:1650]

cv2.circle(img, (int(round(reference[0][0],0)),int(round(reference[0][1],0))), 5, (0,255,0), thickness=-1, lineType=8, shift=0)

cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [51]:
# CHECK DI MISURA
ref = px2meters(reference, K, R, t)
pt = np.array([[112.0, 275.0, 1.0]])
xyz = px2meters(pt, K, R, t)
print('Difference: ' + str((xyz-ref)*100))
repro, _ = cv2.projectPoints(xyz.T, R, t, K, D)
print(repro)

Point is: [[ 32.16716317]
 [ 12.98991198]
 [-41.0146406 ]]
Point is: [[ 32.17045075]
 [ 13.11286614]
 [-41.01872407]]
Difference: [[ 0.32875709]
 [12.29541584]
 [-0.4083469 ]]
[[[104.77696851 272.80744045]]]


In [54]:
img = cv2.imread('/home/optolab/ur2020_workspace/src/telemove/src/calibresult.png')
#immagine antidistorta
img = cv2.flip(img, 0)
img = img[0:900, 520:1650]

reference, _ = cv2.projectPoints(np.array([[0.0, 0.0, 0.0]]), Rd, td, K, D)
reference = reference.flatten()
print('Reference is: ' + str(reference))
reference = np.array([[int(round(reference[0],0)), int(round(reference[1],0)), 1.0]]) #px coordinates

cv2.circle(img, (int(round(reference[0][0],0)),int(round(reference[0][1],0))), 5, (0,255,0), thickness=-1, lineType=8, shift=0)

cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

Reference is: [ 99.75684201 137.88738849]


In [63]:
# CHECK DI MISURA
ref = px2meters(reference, K, Rd, td)
pt = np.array([[99.0, 266.0, 1.0]])
xyz = px2meters(pt, K, Rd, td)
print('Difference: ' + str((xyz-ref)*100))
repro1, _ = cv2.projectPoints(xyz.T, Rd, td, K, D)
repro1 = repro1.ravel()
print(repro1)
repro2, _ = cv2.projectPoints(ref.T, Rd, td, K, D)
repro2 = repro2.ravel()
print(repro2)

Point is: [[ 33.02220338]
 [ 14.16537225]
 [-42.06145587]]
Point is: [[ 33.02317031]
 [ 14.28559119]
 [-42.06390274]]
Difference: [[ 0.09669328]
 [12.02189455]
 [-0.24468652]]
[ 92.22502474 263.91386834]
[ 95.35297    135.99641188]


In [68]:
# errore di reproj
x1 = pt[0][0] - repro1[0]
x2 = reference[0][0] - repro2[0]

y1 = pt[0][1] - repro1[1]
y2 = reference[0][1] - repro2[1]

print(x1)
print(x2)
print(y1)
print(y2)

6.774975257126584
4.647030000340237
2.086131661663387
2.003588124906173


In [69]:
cv2.circle(img, (int(round(repro1[0],0)),int(round(repro1[1],0))), 5, (255,0,0), thickness=-1, lineType=8, shift=0)
cv2.circle(img, (int(round(repro2[0],0)),int(round(repro2[1],0))), 5, (255,0,0), thickness=-1, lineType=8, shift=0)

cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()